### Download dos relatórios em formato PDF, agrupados por safra.


In [5]:
import os
import re
import csv
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import gc
import pandas as pd
from collections import defaultdict

def validar_safra(safra):
    if not re.fullmatch(r'\d{6}', safra):
        raise ValueError(f"Safra '{safra}' deve estar no formato YYYYMM.")
    try:
        datetime.strptime(safra, '%Y%m')
    except ValueError:
        raise ValueError(f"Safra '{safra}' não representa uma data válida.")

def gerar_lista_safras(safra_inicio, safra_fim):
    validar_safra(safra_inicio)
    validar_safra(safra_fim)
    data_inicio = datetime.strptime(safra_inicio, '%Y%m')
    data_fim = datetime.strptime(safra_fim, '%Y%m')
    if data_inicio > data_fim:
        raise ValueError("A safra inicial deve ser anterior ou igual à safra final.")
    safra_atual = data_inicio
    lista = []
    while safra_atual <= data_fim:
        lista.append(safra_atual.strftime('%Y%m'))
        ano = safra_atual.year + (safra_atual.month // 12)
        mes = (safra_atual.month % 12) + 1
        safra_atual = datetime(ano, mes, 1)
    return lista

def baixar_pdf(codigo, safra, url_base, pasta_safra):
    nome_arquivo_pdf = os.path.join(pasta_safra, f"unidade_{codigo}.pdf")
    if os.path.exists(nome_arquivo_pdf):
        return (safra, codigo, "Ignorado", "Arquivo existente.")
    url = f"{url_base}{codigo}&format=pdf"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(nome_arquivo_pdf, 'wb') as f:
            f.write(response.content)
        return (safra, codigo, "Sucesso", "Download OK")
    except requests.RequestException as e:
        return (safra, codigo, "Erro", str(e))

def resumo_por_safra(resultados):
    contagem = defaultdict(lambda: {"Sucesso": 0, "Ignorado": 0, "Erro": 0})
    for safra, _, status, _ in resultados:
        contagem[safra][status] += 1
    return contagem

def download_pdfs(safra_inicio, safra_fim, unidade_inicio, unidade_fim, diretorio, max_threads=5):
    inicio_tempo = time.time()
    lista_safras = gerar_lista_safras(safra_inicio, safra_fim)
    codigos = list(range(unidade_inicio, unidade_fim + 1))
    resultados = []

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        for safra in lista_safras:
            print(f"\n- Processando safra: {safra}")
            pasta_safra = os.path.join(diretorio, safra)
            os.makedirs(pasta_safra, exist_ok=True)
            url_base = (
                f"https://www.tjsp.jus.br/APP/ProdutividadePrimeiraInstancia/Report/RelatorioCorreicao"
                f"?anoMesInicial={safra}01&anoMesFinal={safra}01&codigoUnidade="
            )

            futuros_safra = [executor.submit(baixar_pdf, codigo, safra, url_base, pasta_safra) for codigo in codigos]
            resultados_safra = [future.result() for future in as_completed(futuros_safra)]
            resultados.extend(resultados_safra)

            resumo = resumo_por_safra(resultados_safra)[safra]
            print(f"   - Sucesso: {resumo['Sucesso']}")
            print(f"   - Ignorado: {resumo['Ignorado']}")
            print(f"   - Erros: {resumo['Erro']}")

            gc.collect()

    caminho_csv = os.path.join(diretorio, "relatorio_downloads.csv")
    with open(caminho_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Safra", "CodigoUnidade", "Status", "Mensagem"])
        writer.writerows(resultados)

    duracao_total = time.time() - inicio_tempo
    print(f"\n- Download concluído em {duracao_total:.2f} segundos.")
    print(f"- Relatório CSV salvo em: {caminho_csv}")

    # Resumo final por safra
    resumo_total = resumo_por_safra(resultados)
    print("\n- Resumo geral por safra:")
    for safra, contagem in resumo_total.items():
        print(f"  • Safra {safra}: Sucesso={contagem['Sucesso']} | Ignorado={contagem['Ignorado']} | Erro={contagem['Erro']}")



In [ ]:
download_pdfs(
    safra_inicio="202401",
    safra_fim="202504",
    unidade_inicio=1,
    unidade_fim=5200,
    diretorio="data-raw",
    max_threads=6
)


- Processando safra: 202501
   - Sucesso: 1
   - Ignorado: 5199
   - Erros: 0

- Processando safra: 202502
   - Sucesso: 0
   - Ignorado: 5200
   - Erros: 0

- Processando safra: 202503
   - Sucesso: 0
   - Ignorado: 5200
   - Erros: 0

- Processando safra: 202504
   - Sucesso: 0
   - Ignorado: 5200
   - Erros: 0

- Download concluído em 2.63 segundos.
- Relatório CSV salvo em: demanda_mari\relatorio_downloads.csv

- Resumo geral por safra:
  • Safra 202501: Sucesso=1 | Ignorado=5199 | Erro=0
  • Safra 202502: Sucesso=0 | Ignorado=5200 | Erro=0
  • Safra 202503: Sucesso=0 | Ignorado=5200 | Erro=0
  • Safra 202504: Sucesso=0 | Ignorado=5200 | Erro=0
